<img src='./img/DataTailor_EUMETSAT.png'/>

Copyright (c) 2024 EUMETSAT <br>
License: MIT

<hr>

<a href="./index.ipynb">← Index</a>
<br>
<a href="./1_6_MTG_LI_data_access.ipynb">← Accessing MTG LI products</a><span style="float:right;"><a href="./2_2_Cleaning_the_Data_Tailor_workspace.ipynb">Cleaning the Data Tailor workspace →</a>

## Using the EUMETSAT Data Tailor with EUMDAC

### What will this module teach you?

This module will show you how to:<br>
1. Select a product from the <b>Data Store</b> via EUMDAC library
2. Pass the product to the <b>Data Tailor</b> via EUMDAC library to remotely customise products prior to download
3. Retrieve your customised products

### Use EUMDAC library

In this section we will demonstrate how you can use the Data Tailor web service to customise products you select from the Data Store. This customisation is done <b><u>prior</u></b> to downloading the products, so it is useful for users who wish to refine their data stream. We can do this by using EUMDAC, an EUMETSAT Python library to handle requests and responses of the APIs.

You will find an installation guide, and further information about the usage of EUMDAC, here: https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide

### Authentication

After installing the eumdac we are calling the Token. It takes care of requesting a new value after expiration.

In [1]:
# Import the library EUMDAC
import eumdac
import time
import requests

In [2]:
# Insert your personal key and secret into the single quotes

consumer_key = 'YOUR_CONSUMER_KEY'
consumer_secret = 'YOUR_CONSUMER_SECRET'

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
You can find your personal API credentials here: <a href="https://api.eumetsat.int/api-key/">https://api.eumetsat.int/api-key/</a>
</div>

In [3]:
try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as exc:
    print(f"Error when trying the request to the server: '{exc}'")

This token 'eda98ae8-4156-36b1-9f50-d4226baa589a' expires 2024-09-26 10:15:23.716953


### Selecting a product from the Data Store

For this demonstration, we are going to select a Level 1.5 SEVIRI product (collection ID: EO:EUM:DAT:MSG:HRSEVIRI). This sensor is aboard the geostationary MSG platform, so we do not need to search products by coverage region. SEVIRI products are available every 15 minutes, so we will select the latest product for this example.

In [5]:
datastore = eumdac.DataStore(token)
selected_collection = datastore.get_collection('EO:EUM:DAT:MSG:HRSEVIRI')
latest = selected_collection.search().first()

In [7]:
try:
    print(latest)
except eumdac.datastore.DataStoreError as error:
    print(f"Error related to the data store: '{error.msg}'")
except eumdac.collection.CollectionError as error:
    print(f"Error related to the collection: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

MSG4-SEVI-MSG15-0100-NA-20230313134243.191000000Z-NA


<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
Find more information about EUMDAC errors, their causes and possible solutions, in our knowledge base: <a href="https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide#ID-Exception-handling">https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide#ID-Exception-handling</a>
</div>

## Customising products with the Data Tailor

To customise a product with the Data Tailor, we need to provide following information;
* A product object
* A chain configuration

We already have the product object, 'latest'. Now, we will define our chain configuration.

In [6]:
datatailor = eumdac.DataTailor(token)

# To check if Data Tailor works as expected, we are requesting our quota information
try:
    display(datatailor.quota)
except eumdac.datatailor.DataTailorError as error:
    print(f"Error related to the Data Tailor: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

{'total': 1,
 'data': {'niklasjo': {'disk_quota_active': True,
   'user_quota': 20000.0,
   'space_usage_percentage': 0.0,
   'space_usage': 0.095475,
   'workspace_dir_size': 0.0,
   'log_dir_size': 0.000137,
   'output_dir_size': 0.0,
   'nr_customisations': 1,
   'unit_of_size': 'MB'}}}

#### Defining your own configuration chain
Here we show how to start a customisation with an individual customisation chain. For more info on configuration chains, please go to our [Understanding, Configuring and Using Chains](https://user.eumetsat.int/resources/user-guides/data-store-detailed-guide#ID-Understanding-configuring-and-using-chains) page.

In [26]:
# Defining the chain configuration
chain = eumdac.tailor_models.Chain(
    product='HRSEVIRI',
    format='png_rgb',
    filter={"bands" : ["channel_3","channel_2","channel_1"]},
    projection='geographic',
    roi='west_africa'
)
# roi parameter above is a pre-defined one. It's possible to use "roi=" as follows "{"NSWE" : [37,2,-19,21]}".

In [28]:
# Send the customisation to Data Tailor Web Services
customisation = datatailor.new_customisation(latest, chain)

try:
    print(f"Customisation {customisation._id} started.")
except eumdac.datatailor.DataTailorError as error:
    print(f"Error related to the Data Tailor: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

Customisation d07d8abf started.


After the customisation has started, it's possible to run 'Customisation Loop' below. It checks the status of the customisation until the customisation is completed. While it loops, it prints the status of the job.

In [29]:
status = customisation.status
sleep_time = 10 # seconds

# Customisation Loop
while status:
    # Get the status of the ongoing customisation
    status = customisation.status

    if "DONE" in status:
        print(f"Customisation {customisation._id} is successfully completed.")
        break
    elif status in ["ERROR","FAILED","DELETED","KILLED","INACTIVE"]:
        print(f"Customisation {customisation._id} was unsuccessful. Customisation log is printed.\n")
        print(customisation.logfile)
        break
    elif "QUEUED" in status:
        print(f"Customisation {customisation._id} is queued.")
    elif "RUNNING" in status:
        print(f"Customisation {customisation._id} is running.")
    time.sleep(sleep_time)

Customisation d07d8abf is running.
Customisation d07d8abf is running.
Customisation d07d8abf is successfully completed.


We will show how to download the customised products at the end of the notebook.

#### How to list, create, delete saved configuration chains in Data Tailor

It's possible to create and save a Data Tailor chain. But beware that **you don't need to save/delete chains in your everyday routine**. This optional functionality is meant for users who need to save or delete a chain. The Data Tailor has many pre-defined configurations. You can see all of them with below command.

In [9]:
for chain in datatailor.chains.search():
    print(chain)
    print('---')

Chain(id='aggregated_sst_western_europe', product='GLBSST', format='geotiff', name='Time aggregation of masked SST - Western Europe', description=None, aggregation='time', projection='geographic', roi='western_europe', filter=Filter(id=None, bands=['sea_surface_temperature'], name=None, product=None), quicklook=None, resample_method=None, resample_resolution=None, compression=None)
---
Chain(id='amsal1_aggregation_orbit', product='AMSAL1', format='geotiff', name='Aggregation orbit', description=None, aggregation='orbit', projection=None, roi=None, filter=None, quicklook=None, resample_method=None, resample_resolution=None, compression=None)
---
Chain(id='ascatl1szf_aggregation_orbit', product='ASCATL1SZF', format='geotiff', name='Aggregation orbit', description=None, aggregation='orbit', projection=None, roi=None, filter=None, quicklook=None, resample_method=None, resample_resolution=None, compression=None)
---
Chain(id='ascatl1szo_aggregation_orbit', product='ASCATL1SZO', format='geot

We can see many number of chain configurations are available for each product. These are the default saved configurations, you are able to define your own configuration(s) and save it to the Data Tailor for later use, or submit individual chain configurations with each request.You can see product-specific pre-defined chains with below command.

In [36]:
for chain in datatailor.chains.search(product="HRSEVIRI"):
    print(chain)
    print('---')

Chain(id='hrseviri_visir', product='HRSEVIRI', format='hrit', name='Native to hrit (VISIR)', description=None, aggregation=None, projection=None, roi=None, filter=None, quicklook=None, resample_method=None, resample_resolution=None, compression=None, xrit_segments=None)
---
Chain(id='projection_plate_carree_quicklook', product='HRSEVIRI', format='geotiff', name='Projection Plate-Carree with quick-look', description=None, aggregation=None, projection='geographic', roi='western_europe', filter='hrseviri_natural_color', quicklook=Quicklook(id=None, name=None, resample_method=None, stretch_method='min_max', product=None, format='png_rgb', nodatacolor=None, filter='hrseviri_natural_color', x_size=500), resample_method=None, resample_resolution=None, compression=None, xrit_segments=None)
---
Chain(id='zip_compression', product='HRSEVIRI', format='hrit', name='zip compression', description=None, aggregation=None, projection=None, roi=None, filter=None, quicklook=None, resample_method=None, re

You're able to delete chains using below command.

In [51]:
try:
    datatailor.chains.delete("natural_color_disc")
except eumdac.datatailor.DataTailorError as error:
    print(f"Error related to the Data Tailor: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

Let's see if the "natural_color_disc" is removed from HRSEVIRI pre-defined chains.

In [48]:
for chain in datatailor.chains.search(product="HRSEVIRI"):
    print(chain)
    print('---')

Chain(id='hrseviri_visir', product='HRSEVIRI', format='hrit', name='Native to hrit (VISIR)', description=None, aggregation=None, projection=None, roi=None, filter=None, quicklook=None, resample_method=None, resample_resolution=None, compression=None, xrit_segments=None)
---
Chain(id='natural_color_disc', product='HRSEVIRI', format='geotiff', name='Natural color disc', description=None, aggregation=None, projection=None, roi=None, filter='hrseviri_natural_color', quicklook=None, resample_method=None, resample_resolution=None, compression=None, xrit_segments=None)
---
Chain(id='projection_plate_carree_quicklook', product='HRSEVIRI', format='geotiff', name='Projection Plate-Carree with quick-look', description=None, aggregation=None, projection='geographic', roi='western_europe', filter='hrseviri_natural_color', quicklook=Quicklook(id=None, name=None, resample_method=None, stretch_method='min_max', product=None, format='png_rgb', nodatacolor=None, filter='hrseviri_natural_color', x_size=5

Let's create that chain and check the HRSEVIRI chains again.

In [52]:
chain = eumdac.tailor_models.Chain(id='natural_color_disc', 
      product='HRSEVIRI', 
      format='geotiff',
      name='Natural color disc',
      filter='hrseviri_natural_color')

try:
    datatailor.chains.create(chain)
except eumdac.datatailor.DataTailorError as error:
    print(f"Data Tailor Error", error)
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

In [53]:
for chain in datatailor.chains.search(product="HRSEVIRI"):
    try:
        print(chain)
    except eumdac.datatailor.DataTailorError as error:
        print(f"Data Tailor Error", error)
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")
    print('---')

Chain(id='hrseviri_visir', product='HRSEVIRI', format='hrit', name='Native to hrit (VISIR)', description=None, aggregation=None, projection=None, roi=None, filter=None, quicklook=None, resample_method=None, resample_resolution=None, compression=None, xrit_segments=None)
---
Chain(id='natural_color_disc', product='HRSEVIRI', format='geotiff', name='Natural color disc', description=None, aggregation=None, projection=None, roi=None, filter='hrseviri_natural_color', quicklook=None, resample_method=None, resample_resolution=None, compression=None, xrit_segments=None)
---
Chain(id='projection_plate_carree_quicklook', product='HRSEVIRI', format='geotiff', name='Projection Plate-Carree with quick-look', description=None, aggregation=None, projection='geographic', roi='western_europe', filter='hrseviri_natural_color', quicklook=Quicklook(id=None, name=None, resample_method=None, stretch_method='min_max', product=None, format='png_rgb', nodatacolor=None, filter='hrseviri_natural_color', x_size=5

Let's trigger a cutomisation with the saved chain. To use one of the saved configurations, you need to call them by their ID:

In [56]:
natural_color_disc = datatailor.chains.read('natural_color_disc')

Now, we have to define and start the customisation by calling the product and chain:

In [57]:
# Send the customisation to Data Tailor Web Services
customisation = datatailor.new_customisation(latest, chain=natural_color_disc)

try:
    print(f"Customisation {customisation._id} started.")
except eumdac.datatailor.CustomisationError as error:
    print(f"Data Tailor Error", error)
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

Customisation cf796feb started.


After the customisation has started, it's possible to run 'Customisation Loop' below. It checks the status of the customisation until the customisation is completed. While it loops, it prints the status of the job.

In [17]:
status = customisation.status
sleep_time = 10 # seconds

# Customisation Loop
while status:
    # Get the status of the ongoing customisation
    status = customisation.status

    if "DONE" in status:
        print(f"Customisation {customisation._id} is successfully completed.")
        break
    elif status in ["ERROR","FAILED","DELETED","KILLED","INACTIVE"]:
        print(f"Customisation {customisation._id} was unsuccessful. Customisation log is printed.\n")
        print(customisation.logfile)
        break
    elif "QUEUED" in status:
        print(f"Customisation {customisation._id} is queued.")
    elif "RUNNING" in status:
        print(f"Customisation {customisation._id} is running.")
    time.sleep(sleep_time)

Customisation dc14d444 is running.
Customisation dc14d444 is running.
Customisation dc14d444 is successfully completed.


#### Create a chain and trigger a customisation

Let's finally create a customisation chain from scratch, send it to DTWS and download the output.

In [60]:
chain = eumdac.tailor_models.Chain(
    id='hrseviri_nc_west-africa',
    name='Native to PNG of West Africa',
    description='Convert a SEVIRI Native product to PNG with subsetting the region of West Africa',
    product='HRSEVIRI',
    format='png_rgb',
    filter='hrseviri_natural_color',
    projection='geographic',
    roi='west_africa')

try:
    datatailor.chains.create(chain)
except eumdac.datatailor.DataTailorError as error:
    print(f"Data Tailor Error", error)
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

Let's check if saved chain is there.

In [64]:
for chain in datatailor.chains.search(product="HRSEVIRI"):
    try:
        print(chain)
    except eumdac.datatailor.DataTailorError as error:
        print(f"Data Tailor Error", error)
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")
    print('---')

Chain(id='hrseviri_visir', product='HRSEVIRI', format='hrit', name='Native to hrit (VISIR)', description=None, aggregation=None, projection=None, roi=None, filter=None, quicklook=None, resample_method=None, resample_resolution=None, compression=None, xrit_segments=None)
---
Chain(id='natural_color_disc', product='HRSEVIRI', format='geotiff', name='Natural color disc', description=None, aggregation=None, projection=None, roi=None, filter='hrseviri_natural_color', quicklook=None, resample_method=None, resample_resolution=None, compression=None, xrit_segments=None)
---
Chain(id='projection_plate_carree_quicklook', product='HRSEVIRI', format='geotiff', name='Projection Plate-Carree with quick-look', description=None, aggregation=None, projection='geographic', roi='western_europe', filter='hrseviri_natural_color', quicklook=Quicklook(id=None, name=None, resample_method=None, stretch_method='min_max', product=None, format='png_rgb', nodatacolor=None, filter='hrseviri_natural_color', x_size=5

Now, we can start the customisation by calling the product and chain:

In [20]:
hrseviri_nc_west_africa = datatailor.chains.read('hrseviri_nc_west-africa')

# Send the customisation to Data Tailor Web Services
customisation = datatailor.new_customisation(latest, chain=hrseviri_nc_west_africa)

try:
    print(f"Customisation {customisation._id} started.")
except eumdac.datatailor.CustomisationError as error:
    print(f"Data Tailor Error", error)
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

Customisation a7fcf12c started.


Let's use our loop to check if the job was successful.

In [21]:
status = customisation.status
sleep_time = 10 # seconds

# Customisation Loop
while status:
    # Get the status of the ongoing customisation
    status = customisation.status

    if "DONE" in status:
        print(f"Customisation {customisation._id} is successfully completed.")
        break
    elif status in ["ERROR","FAILED","DELETED","KILLED","INACTIVE"]:
        print(f"Customisation {customisation._id} was unsuccessful. Customisation log is printed.\n")
        print(customisation.logfile)
        break
    elif "QUEUED" in status:
        print(f"Customisation {customisation._id} is queued.")
    elif "RUNNING" in status:
        print(f"Customisation {customisation._id} is running.")
    time.sleep(sleep_time)

Customisation a7fcf12c is running.
Customisation a7fcf12c is running.
Customisation a7fcf12c is running.
Customisation a7fcf12c is running.
Customisation a7fcf12c is running.
Customisation a7fcf12c is successfully completed.


### Downloading customised data

In order to download the PNGs we just generated with the customisation, we need some standard Python libraries that we have to import.

In [11]:
import fnmatch
import shutil

We just want to get the PNG file, so we are creating a filter for it:

In [23]:
png, = fnmatch.filter(customisation.outputs, '*.png')

Now, we can start the download with the following:

In [24]:
jobID= customisation._id

print(f"Dowloading the PNG output of the customisation {jobID}")

try:
    with customisation.stream_output(png,) as stream, \
            open(stream.name, mode='wb') as fdst:
        shutil.copyfileobj(stream, fdst)
    print(f"Dowloaded the PNG output of the customisation {jobID}")
except eumdac.datatailor.CustomisationError as error:
    print(f"Data Tailor Error", error)
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

Dowloading the PNG output of the customisation a7fcf12c
Dowloaded the PNG output of the customisation a7fcf12c


### Clearing customisations from the Data Tailor

The Data Tailor Web Service has a 20 Gb limit, so it's **important** to clear old customisations. To delete the customisation we have just created, simply call the delete function:

In [25]:
try:
    customisation.delete()
except eumdac.datatailor.CustomisationError as exc:
    print("Customisation Error:", exc)
except requests.exceptions.RequestException as error:
    print("Unexpected error:", error)

More and complete information about clearing customisations from the Data Tailor can be found in our <a href="./2_Cleaning_the_Data_Tailor_workspace.ipynb"> second notebook.</a>

## Aggregating multiple products using EUMDAC Python

As shown in the example above, using `datatailor.new_customisation` produces one job for one product. As a special case, there is a new interface `datatailor.new_customisations` that can be used to create a single job for multiple products when the tailoring chain contains an aggregation operation. This section provides an example on using this interface to aggregate multiple products.

We first set the collection and other filtering parameters and list the products to be aggregated.

In [7]:
import datetime

# Selecting the collection and product.
selected_collection = datastore.get_collection('EO:EUM:DAT:METOP:GLB-SST-NC')

# Set sensing start and end time
start = datetime.datetime(2023, 10, 1, 9, 0)
end = datetime.datetime(2023, 10, 3, 9, 0)

selected_products = selected_collection.search(dtstart = start, dtend = end)
print(f'Found Datasets: {selected_products.total_results} datasets for the given time range')
for product in selected_products:
    print(str(product))

Found Datasets: 5 datasets for the given time range
S-OSI_-FRA_-MTOP-GLBSST_FIELD-202310031200Z
S-OSI_-FRA_-MTOP-GLBSST_FIELD-202310030000Z
S-OSI_-FRA_-MTOP-GLBSST_FIELD-202310021200Z
S-OSI_-FRA_-MTOP-GLBSST_FIELD-202310020000Z
S-OSI_-FRA_-MTOP-GLBSST_FIELD-202310011200Z


We create the tailoring chain with our desired customisation operations that include an aggregation operation.

In [8]:
# Creating custom tailoring chain.
chain_agg = eumdac.tailor_models.Chain(product='GLBSST',
                                       format='geotiff',
                                       aggregation='time', # Aggregation in chain!
                                       filter={"bands":['sea_surface_temperature']})

We create a new job with all the selected products using the new_customisations interface. Since the tailoring chain contains an aggregation operation, the interface would return a list with a single element corresponding to the single job. We finally extract the list element.

In [9]:
# The `new_customisations` interface has the ability to return
# single job for multiple products when the tailoring chain
# contains an aggregation operation.
customisation = datatailor.new_customisations(selected_products, chain=chain_agg)
# Since there is an aggregation operation in the tailoring chain,
# a single job will be created. Consequently, `customisation` is
# a list with a single element.
customisation = customisation[0]

We define a time duration for the interval with which to query the status of the customisation. Once the customisation has finished, we download the result in the usual manner as described previously.

In [12]:
sleep_time = 10 # seconds
while True:
    status = customisation.status
    if "DONE" in status:
        print(f"Customisation {customisation._id} is successfully completed.")
        print(f"Downloading the output of the customisation {customisation._id}")
        cust_files = fnmatch.filter(customisation.outputs, '*')[0]
        with customisation.stream_output(cust_files) as stream, open(stream.name, mode='wb') as fdst:
            shutil.copyfileobj(stream, fdst)
        print(f"Download finished for customisation {customisation._id}.")
        break
    elif status in ["ERROR", "FAILED", "DELETED", "KILLED", "INACTIVE"]:
        print(f"Customisation {customisation._id} was unsuccessful. Customisation log is printed.\n")
        print(customisation.logfile)
        break
    elif "QUEUED" in status:
        print(f"Customisation {customisation._id} is queued.")
    elif "RUNNING" in status:
        print(f"Customisation {customisation._id} is running.")
    time.sleep(sleep_time)

Customisation 13c31242 is successfully completed.
Download finished for customisation 13c31242.


This ends our example on how to perform remote customisations on Data Store products using the Data Tailor. Feel free to adapt this script to search for your own products of interest, and customise with your chains as required. Guidance on chain customisation can be found on our <a href="https://user.eumetsat.int/resources/user-guides/data-store-detailed-guide#ID-Understanding-configuring-and-using-chains">Understanding, Configuring and Using Chains</a> page. If you need further help, you can contact us using the buttons at the bottom of the page.

<a href="./index.ipynb">← Index</a>
<br>
<a href="./1_5_MTG_data_access.ipynb">← Downloading and visualising MTG FCI products</a><span style="float:right;"><a href="./2_2_Cleaning_the_Data_Tailor_workspace.ipynb">Cleaning the Data Tailor workspace →</a>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE.TXT">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/data-services/eumdac_data_store">View on GitLab</a> | <a href="https://classroom.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact</a></span></p>